Version Two: Ongoing

In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [14]:
import pandas as pd
from gensim.models.phrases import Phrases, Phraser
from time import time 
import multiprocessing
from gensim.models import Word2Vec

In [3]:
df = pd.read_csv('Data/data.csv')
df.head()

,id,text
0,0,taarifa hiyo ilisema kuwa ongezeko la joto la ...
1,1,aidha ilisema kuwa mwelekeo wa kupungua kwa jo...
2,2,mwelekeo wa mvua wa septemba hadi desemba ishi...
3,3,ilifafanua kuwa msimu wa vuli UNK maeneo ambay...
4,4,katika maeneo hayo mvua zinatarajiwa kunyesha ...


In [13]:
df_clean = df
from nltk.tokenize import RegexpTokenizer
t = time()

tokenizer = RegexpTokenizer(r'\w+')
df_clean['clean'] = df_clean['text'].astype('str') 
df_clean.dtypes

df_clean["tokens"] = df_clean["clean"].apply(tokenizer.tokenize)
# delete Stop Words

print('Time to tokenize everything: {} mins'.format(round((time() - t) / 60, 2)))
df_clean.head()

Time to tokenize everything: 0.0 mins


,id,text,clean,tokens
0,0,taarifa hiyo ilisema kuwa ongezeko la joto la ...,taarifa hiyo ilisema kuwa ongezeko la joto la ...,"[taarifa, hiyo, ilisema, kuwa, ongezeko, la, j..."
1,1,aidha ilisema kuwa mwelekeo wa kupungua kwa jo...,aidha ilisema kuwa mwelekeo wa kupungua kwa jo...,"[aidha, ilisema, kuwa, mwelekeo, wa, kupungua,..."
2,2,mwelekeo wa mvua wa septemba hadi desemba ishi...,mwelekeo wa mvua wa septemba hadi desemba ishi...,"[mwelekeo, wa, mvua, wa, septemba, hadi, desem..."
3,3,ilifafanua kuwa msimu wa vuli UNK maeneo ambay...,ilifafanua kuwa msimu wa vuli UNK maeneo ambay...,"[ilifafanua, kuwa, msimu, wa, vuli, UNK, maene..."
4,4,katika maeneo hayo mvua zinatarajiwa kunyesha ...,katika maeneo hayo mvua zinatarajiwa kunyesha ...,"[katika, maeneo, hayo, mvua, zinatarajiwa, kun..."


In [15]:
#WORD2VEC()
cores = multiprocessing.cpu_count() # Count the number of cores in a computer, important for a parameter of the model
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=200,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

#BUILD_VOCAB()
t = time()
w2v_model.build_vocab(df_clean["tokens"], progress_per=1000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

#TRAIN()
w2v_model.train(df_clean["tokens"], total_examples=w2v_model.corpus_count, epochs=10000, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.0 mins
Time to train the model: 0.28 mins


In [26]:
w2v_model.vector_size

300